In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy 
import scipy as sp

from scipy import stats,signal
import statsmodels.api as sm
import math

from scipy.fft import fft, ifft 

from scipy.signal import medfilt


col = ['X','Y', 'Z']

In [24]:
# import the median filter function
# return the med-filtered signal: numpy array 1D
def median_filter(signal):       # input: numpy array 1D (one column)
    array=np.array(signal) 
    
    #applying the median filter
    med_filtered=scipy.signal.medfilt(array, kernel_size=3) # applying the median filter order3(kernel_size=3)
    return  med_filtered 



## Low-path Butterworth Filter
def butter_lowpass(cutoff, nyq_freq, order): ##CORRECT
    normal_cutoff = float(cutoff) / nyq_freq
    b, a = signal.butter(order, normal_cutoff, btype='lowpass')
    return b, a

def butter_lowpass_filter(data, cutoff_freq, nyq_freq, order): ##CORRECT
    b, a = butter_lowpass(cutoff_freq, nyq_freq, order=order)
    y = signal.filtfilt(b, a, data) 
    return y



## Derivative
def jerk_one_signal(signal):
    dt=0.02 # dt=1/50=0.02s time duration between two rows
#     return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])
    return signal.diff()


### Magnitude
def mag(df):
    result = []
    for i in range(df.shape[0]):
        x, y, z = df.iloc[i,:].values
        mag = np.sqrt(x**2 + y**2 + z**2)
        result.append(mag)
    return result


## FFT
from scipy import fftpack # import fftpack to use all fft functions
from numpy.fft import *

##################### fast_fourier_transform_one_signal #################
# Inputs: time signal 1D array
# Output: amplitude of fft components 1D array having the same lenght as the Input
def fast_fourier_transform_one_signal(t_signal):
    # apply fast fourrier transform to the t_signal
    complex_f_signal= fftpack.fft(t_signal)
    #compute the amplitude each complex number
    amplitude_f_signal=np.abs(complex_f_signal)
    # return the amplitude
    return amplitude_f_signal

In [25]:
import serial, time

def Arduino(n):
    ser = serial.Serial(port='/dev/cu.usbmodem1411', baudrate=38400)

    if ser.isOpen():
        print("{} connected!".format(ser.port))
        try:
            start = time.time()
            
            gryo = {'X':[], 'Y': [], 'Z': []}
            acc = {'X':[], 'Y': [], 'Z': []}
            while True:  
                try:
                    line_byte = ser.readline().strip().decode("UTF-8") 
#                     print(line_byte)
                except: 
                    continue
                    
                if 'Gryo' in line_byte:
                    try:
                        line = line_byte.split('\t')
                        gryo['X'].append(line[1])
                        gryo['Y'].append(line[2])
                        gryo['Z'].append(line[3])
                    except:
                        continue
                if 'Acc' in line_byte:
                    try:
                        line = line_byte.split('\t')
                        acc['X'].append(line[1])
                        acc['Y'].append(line[2])
                        acc['Z'].append(line[3])
                       
                    except:
                        continue
                    
                    if time.time() - start > n:
                        break
            return gryo, acc

        except KeyboardInterrupt:
            print("KeyboardInterrupt has been caught.")

In [26]:
gryo, acc = Arduino(2.56 * 2)


SerialException: [Errno 2] could not open port /dev/cu.usbmodem1411: [Errno 2] No such file or directory: '/dev/cu.usbmodem1411'

In [27]:
acc_base = pd.DataFrame(acc, columns = col)
gryo_base = pd.DataFrame(acc, columns = col)

for i in col:
    acc_base[[str(i)]] = acc_base[[i]].astype(float)
    gryo_base[[str(i)]] = gryo_base[[i]].astype(float)
    
  
# for i in col:
#     acc_base[str(i)] = np.squeeze(np.array(acc_base[i]) / 16384)

NameError: name 'acc' is not defined

In [28]:
acc_base.plot()

NameError: name 'acc_base' is not defined

In [29]:
acc_base.shape

NameError: name 'acc_base' is not defined

In [ ]:
acc_base = acc_base[:129]

In [ ]:
gryo_base = gryo_base[:129]

In [ ]:
acc =acc_base

In [ ]:
# med_gryo = pd.DataFrame(columns=col)
med_acc = pd.DataFrame(columns=col)
for i in col:
#     med_gryo[i] = median_filter(gryo[i])
    med_acc[i] = median_filter(acc[i])


##############################################################
#Butterworth Filter
##############################################################

sample_rate = 50                    # 50Hz
signal_length = 2.56 * sample_rate  # 2.56 seconds
freq_20 = 20.0                      # 20Hz
freq_03 = 0.3                       # 0.3HZ
nyq_freq = sample_rate / 2          # 25Hz


# tGyro = pd.DataFrame(columns=col)
tAcc = pd.DataFrame(columns=col)

for i in col:
#     tGyro[i] = butter_lowpass_filter(med_gryo[i], freq_20, nyq_freq, 3)
    tAcc[i] = butter_lowpass_filter(med_acc[i], freq_20, nyq_freq, 3)

# tBodyGyro = tGyro

tGravityAcc = pd.DataFrame(columns=col)
tBodyAcc = pd.DataFrame(columns=col)

for i in col:
    tGravityAcc[i] = butter_lowpass_filter(tAcc[i], 0.3, nyq_freq, 1)
    tBodyAcc[i] = tAcc[i] - tGravityAcc[i] 




# tBodyAccJerk = pd.DataFrame(columns=col)
# tBodyGyroJerk = pd.DataFrame(columns=col)


# for i in col:
#     #Derivative
#     tBodyAccJerk[i] = jerk_one_signal(tBodyAcc[i])
#     tBodyGyroJerk[i] = jerk_one_signal(tBodyGyro[i])

# tGravityAcc = tBodyGyro[:-1]
# tBodyAcc = tBodyAcc[:-1]

# tBodyGyro = tBodyGyro[:-1]

In [ ]:
tGravityAcc.plot()

In [ ]:
def mag_3_signals(x,y,z): # Euclidian magnitude
    return [math.sqrt((x[i]**2+y[i]**2+z[i]**2)) for i in range(len(x))]
grav_acc_mag = mag_3_signals(tGravityAcc["X"], tGravityAcc['Y'], tGravityAcc['Z'])

print('mean value = '+str(np.array(grav_acc_mag).mean())[0:5]+ ' g')

In [ ]:
acc = pd.read_csv('acc_test.csv', header = None, delimiter = ' ', names = col)
acc.describe()


In [38]:
def main(acc, gryo):
    
    med_gryo = pd.DataFrame(columns=col)
    med_acc = pd.DataFrame(columns=col)
    for i in col:
        med_gryo[i] = median_filter(gryo[i])
        med_acc[i] = median_filter(acc[i])
        
     
    ##############################################################
    #Butterworth Filter
    ##############################################################
    
    sample_rate = 50                    # 50Hz
    signal_length = 2.56 * sample_rate  # 2.56 seconds
    freq_20 = 20.0                      # 20Hz
    freq_03 = 0.3                       # 0.3HZ
    nyq_freq = sample_rate / 2          # 25Hz
    
    
    tGyro = pd.DataFrame(columns=col)
    tAcc = pd.DataFrame(columns=col)

    for i in col:
        tGyro[i] = butter_lowpass_filter(med_gryo[i], freq_20, nyq_freq, 3)
        tAcc[i] = butter_lowpass_filter(med_acc[i], freq_20, nyq_freq, 3)

    tBodyGyro = tGyro

    tGravityAcc = pd.DataFrame(columns=col)
    tBodyAcc = pd.DataFrame(columns=col)

    for i in col:
        tGravityAcc[i] = butter_lowpass_filter(tAcc[i], 0.3, nyq_freq, 1)
        tBodyAcc[i] = tAcc[i] - tGravityAcc[i] 



    
    tBodyAccJerk = pd.DataFrame(columns=col)
    tBodyGyroJerk = pd.DataFrame(columns=col)


    for i in col:
        #Derivative
        tBodyAccJerk[i] = jerk_one_signal(tBodyAcc[i])
        tBodyGyroJerk[i] = jerk_one_signal(tBodyGyro[i])
        
    tBodyAccJerk.fillna(0, inplace = True)
    tBodyGyroJerk.fillna(0, inplace = True)
        
#     tGravityAcc = tBodyGyro[:-1]
#     tBodyAcc = tBodyAcc[:-1]
     
#     tBodyGyro = tBodyGyro[:-1]
    
    ##############################################################
    # Magnitude
    ##############################################################
    
    
    tBodyAccMag = pd.DataFrame(columns=['tBodyAccMag'])
    tBodyAccMag['tBodyAccMag'] =  mag(tBodyAcc)

    tGravityAccMag = pd.DataFrame(columns=['tGravityAccMag'])
    tGravityAccMag['tGravityAccMag'] = mag(tGravityAcc)

    tBodyAccJerkMag = pd.DataFrame(columns=['tBodyAccJerkMag'])
    tBodyAccJerkMag['tBodyAccJerkMag'] = mag(tBodyAccJerk)

    tBodyGyroMag = pd.DataFrame(columns=['tBodyGyroMag'])
    tBodyGyroMag['tBodyGyroMag'] = mag(tBodyGyro)

    tBodyGyroJerkMag = pd.DataFrame(columns=['tBodyGyroJerkMag'])
    tBodyGyroJerkMag['tBodyGyroJerkMag'] = mag(tBodyGyroJerk)
    
    
    ##############################################################
    # Fourier Transform
    ##############################################################
    
    fBodyAcc= pd.DataFrame(columns=col)
    fBodyAccJerk = pd.DataFrame(columns=col)
    fBodyGyro = pd.DataFrame(columns=col)

    for i in col:
        fBodyAcc[i] = fast_fourier_transform_one_signal(tBodyAcc[i].values)
        fBodyAccJerk[i] = fast_fourier_transform_one_signal(tBodyAccJerk[i].values)    
        fBodyGyro[i] = fast_fourier_transform_one_signal(tBodyGyro[i].values)    

    fBodyAccMag = pd.DataFrame(columns=['fBodyAccMag'])
    fBodyAccMag['fBodyAccMag'] = fast_fourier_transform_one_signal(tBodyAccMag['tBodyAccMag'].values)  

    fBodyAccJerkMag = pd.DataFrame(columns=['fBodyAccJerkMag'])
    fBodyAccJerkMag['fBodyAccJerkMag'] = fast_fourier_transform_one_signal(tBodyAccJerkMag['tBodyAccJerkMag'].values)   

    fBodyGyroMag = pd.DataFrame(columns=['fBodyGyroMag'])
    fBodyGyroMag['fBodyGyroMag'] = fast_fourier_transform_one_signal(tBodyGyroMag['tBodyGyroMag'].values) 

    fBodyGyroJerkMag = pd.DataFrame(columns=['fBodyGyroJerkMag'])
    fBodyGyroJerkMag['fBodyGyroJerkMag'] = fast_fourier_transform_one_signal(tBodyGyroJerkMag['tBodyGyroJerkMag'].values) 

    
    return tBodyAcc, tGravityAcc, tBodyAccJerk, tBodyGyro, tBodyGyroJerk, \
           tBodyAccMag, tGravityAccMag,tBodyAccJerkMag, tBodyGyroMag, tBodyGyroJerkMag, \
           fBodyAcc, fBodyAccJerk, fBodyGyro, \
           fBodyAccMag,fBodyAccJerkMag, fBodyGyroMag,fBodyGyroJerkMag

        

In [39]:
# Mean Calculation
def mean(x):
    mean_x = np.mean(x)
    return mean_x

# Standard Deviation Calculation
def std_dev(x):
    std_x = np.std(x)
    return std_x

from statsmodels.robust import mad as median_deviation
# Median Absolute Deviation Calculation
def mad(x):
    mad_x = median_deviation(x)
    return mad_x

# Min Calculation
def min_value(x):
    return min(x)

# Max Calculation
def max_value(x):
    return max(x)

from scipy.stats import iqr as IQR
# Inter-quartile range Calculation
def inter_quartile_range(x):
    iqr_x = IQR(x,  axis = 0)
    return iqr_x

# Entropy
from scipy.stats import entropy # import the entropy function
def entropy_axial(x): 
    entropy_vector = entropy(abs(x))
    return entropy_vector

In [40]:
##################################################################################
# 40 FEATURE CALCULATION
##################################################################################

def t_sma_axial(df):
    array=np.array(df)
    sma_axial=float(abs(array).sum())/float(3) # sum of areas under each signal
    return sma_axial # return sma value

# Energy measure. Sum of the squares divided by the number of values. 
def energy_measure(x):
    em_x = np.sum(np.square(x))
    return em_x


def _arburg2(X, order):
    """This version is 10 times faster than arburg, but the output rho is not correct.
    returns [1 a0,a1, an-1]
    """
    x = np.array(X)
    N = len(x)

    if order == 0.:
        raise ValueError("order must be > 0")

    # Initialisation
    # ------ rho, den
    rho = sum(abs(x)**2.) / N  # Eq 8.21 [Marple]_
    den = rho * 2. * N

    # ------ backward and forward errors
    ef = np.zeros(N, dtype=complex)
    eb = np.zeros(N, dtype=complex)
    for j in range(0, N):  # eq 8.11
        ef[j] = x[j]
        eb[j] = x[j]

    # AR order to be stored
    a = np.zeros(1, dtype=complex)
    a[0] = 1
    # ---- rflection coeff to be stored
    ref = np.zeros(order, dtype=complex)

    E = np.zeros(order+1)
    E[0] = rho

    for m in range(0, order):
        # print m
        # Calculate the next order reflection (parcor) coefficient
        efp = ef[1:]
        ebp = eb[0:-1]
        # print efp, ebp
        num = -2. * np.dot(ebp.conj().transpose(), efp)
        den = np.dot(efp.conj().transpose(),  efp)
        den += np.dot(ebp,  ebp.conj().transpose())
        ref[m] = num / den

        # Update the forward and backward prediction errors
        ef = efp + ref[m] * ebp
        eb = ebp + ref[m].conj().transpose() * efp

        # Update the AR coeff.
        a.resize(len(a)+1)
        a = a + ref[m] * np.flipud(a).conjugate()

        # Update the prediction error
        E[m+1] = np.real((1 - ref[m].conj().transpose() * ref[m])) * E[m]
        # print 'REF', ref, num, den
    return a, E[-1], ref

def t_arburg_axial(df):
    # converting signals to 1D numpy arrays for efficiency
    array_X=np.array(df[df.columns[0]])
    array_Y=np.array(df[df.columns[1]])
    array_Z=np.array(df[df.columns[2]])
    
    AR_X = list(_arburg2(array_X,4)[0][1:].real) # list contains real parts of all 4th coefficients generated from signal_X
    AR_Y = list(_arburg2(array_Y,4)[0][1:].real) # list contains real parts of all 4th coefficients generated from signal_Y
    AR_Z = list(_arburg2(array_Z,4)[0][1:].real) # list contains real parts of all 4th coefficients generated from signal_Z
    
    # selecting [AR1 AR2 AR3 AR4] real components for each axis concatenate them in one vector
    AR_vector= AR_X + AR_Y+ AR_Z
    
    # AR_vector contains 12 values 4values per each axis 
    return AR_vector


# correlation(): correlation coefficient between two signals
def coor(signal):
    corr = signal.corr()
    x_y = corr.loc['X','Y']
    x_z = corr.loc['X','Z']
    y_z = corr.loc['Y','Z']
    
    return x_y, x_z, y_z



#################################################################################
# 13 FEATURE
# T VALUES
#################################################################################

# sma: signal magnitude area
def t_sma_mag(mag_column):
    array=np.array(mag_column)
    sma_mag=float(abs(array).sum())# signal magnitude area of one mag column
    return sma_mag

# arburg: auto regression coefficients using the burg method
def t_arburg_mag(mag_column):
    array = np.array(mag_column)    
    AR_vector= list(_arburg2(array,4)[0][1:].real) # AR1, AR2, AR3, AR4 of the mag column
    return AR_vector

##############################################################################
#80 FEATURE EXTRACTION FORMULA 
# FFT RESULTS
##############################################################################
def f_sma_axial(df):
    array=np.array(df)
    sma_value=float((abs(array)/math.sqrt(128)).sum())/float(3) # sma value of 3-axial f_signals
    return sma_value

# energy
def f_energy_axial(x):
    em_x = np.mean(np.square(x))
    return em_x
    
####### Max Inds and Mean_Freq Functions#######################################                                

# max_Inds
def f_max_Inds_axial(df):
    # built frequencies list (each column contain 128 value)
    # duration between each two successive captures is 0.02 s= 1/50hz
    freqs=sp.fftpack.fftfreq(128, d=0.02) 
    array=np.array(df)
    max_Inds_X =freqs[array[1:65,0].argmax()+1] # return the frequency related to max value of f_signal X
    max_Inds_Y =freqs[array[1:65,1].argmax()+1] # return the frequency related to max value of f_signal Y
    max_Inds_Z =freqs[array[1:65,2].argmax()+1] # return the frequency related to max value of f_signal Z
    max_Inds_vector= [max_Inds_X,max_Inds_Y,max_Inds_Z]# put those frequencies in a list
    return max_Inds_vector

# mean freq()
def f_mean_Freq_axial(df):
    # built frequencies list (each column contain 128 value)
    # duration between each two successive captures is 0.02 s= 1/50hz
    freqs=sp.fftpack.fftfreq(128, d=0.02) 
    array=np.array(df)
    
   
    
    # sum of( freq_i * f_signal[i])/ sum of signal[i]
    mean_freq_X = np.dot(freqs,array[:,0]).sum() / float(array[:,0].sum()) #  frequencies weighted sum using f_signalX
    mean_freq_Y = np.dot(freqs,array[:,1]).sum() / float(array[:,1].sum()) #  frequencies weighted sum using f_signalY 
    mean_freq_Z = np.dot(freqs,array[:,2]).sum() / float(array[:,2].sum()) #  frequencies weighted sum using f_signalZ
    mean_freq_vector=[mean_freq_X,mean_freq_Y,mean_freq_Z] # vector contain mean frequencies[X,Y,Z]
    return  mean_freq_vector

# skewness(): skewness of the frequency domain signal 
def skewness(x):
    skew_x=stats.skew(x)
    return skew_x

# kurtosis(): kurtosis of the frequency domain signal 
def kurtosis(x):
    kurt_x = stats.kurtosis(x,fisher=False)
    return kurt_x

def f_one_band_energy(f_signal,band): # f_signal is one column in frequency axial signals in f_window
    # band: is one tuple in B1 ,B2 or B3 
    f_signal_bounded = f_signal[band[0]:band[1]] # select f_signal components included in the band
    energy_value=float((f_signal_bounded**2).sum()/float(len(f_signal_bounded))) # energy value of that band
    return energy_value



######################################################################### 
#Functions used to generate frequency magnitude features
#13 FEATURE
#########################################################################

# sma
def f_sma_mag(mag_column):
    array=np.array(mag_column)
    sma_value=float((abs(array)/math.sqrt(len(mag_column))).sum()) # sma of one mag f_signals
    return sma_value

# energy
def f_energy_mag(mag_column):
    array=np.array(mag_column)
    # spectral energy value
    energy_value=float((array**2).sum()/float(len(array))) # energy value of one mag f_signals
    return energy_value


# Max Inds and Mean_Freq Functions
# max_Inds
def f_max_Inds_mag(mag_column):
    freqs=sp.fftpack.fftfreq(128, d=0.02) 
    array=np.array(mag_column)
    max_Inds_value =float(freqs[array[1:65].argmax()+1]) # freq value related with max component
    return max_Inds_value

# mean freq()
def f_mean_Freq_mag(mag_column):
    freqs=sp.fftpack.fftfreq(128, d=0.02) 
    array=np.array(mag_column)
    mean_freq_value = float(np.dot(freqs,array).sum() / float(array.sum())) # weighted sum of one mag f_signal
    return  mean_freq_value


#Skewness & Kurtosis
from scipy.stats import skew           # skewness
def f_skewness_mag(mag_column):
    array=np.array(mag_column)
    skew_value     = float(skew(array)) # skewness value of one mag f_signal
    return skew_value

from scipy.stats import kurtosis       # kurtosis
def f_kurtosis_mag(mag_column):
    array=np.array(mag_column)
    kurtosis_value = float(kurtosis(array)) # kurotosis value of on mag f_signal
    return kurtosis_value




#################################################################################################
#LAST FEATURES
#ANGLES
############### Angles Functions ####################################
from math import acos # inverse of cosinus function
from math import sqrt # square root function

########Euclidian magnitude 3D############
def magnitude_vector(vector3D): # vector[X,Y,Z]
    return sqrt((vector3D**2).sum()) # eulidian norm of that vector

###########angle between two vectors in radian ###############
def angle(vector1, vector2):
    vector1_mag=magnitude_vector(vector1) # euclidian norm of V1
    vector2_mag=magnitude_vector(vector2) # euclidian norm of V2
   
    scalar_product=np.dot(vector1,vector2) # scalar product of vector 1 and Vector 2
    cos_angle=scalar_product/float(vector1_mag*vector2_mag) # the cosinus value of the angle between V1 and V2
    
    # just in case some values were added automatically
    if cos_angle>1:
        cos_angle=1
    elif cos_angle<-1:
        cos_angle=-1
    
    angle_value=float(acos(cos_angle)) # the angle value in radian
    return angle_value # in radian.



# Feature Collections

In [41]:
def feature_40(signal, name):
    l = [ 'mean_X', 'mean_Y', 'mean_Z',
         'std_X','std_Y', 'std_Z',
         'mad_X','mad_Y', 'mad_Z',
         'max_X','max_Y', 'max_Z',
         'min_X','min_Y', 'min_Z',
         'sma',
         'energy_X', 'energy_Y','energy_Z',
         'iqr_X','iqr_Y','iqr_Z',
         'entropy_X', 'entropy_Y', 'entropy_Z',
         'X_1','X_2','X_3','X_4','Y_1','Y_2','Y_3','Y_4', 'Z_1','Z_2','Z_3','Z_4',
         'correlation_x_y','correlation_x_z', 'correlation_y_z'
         ]
    col = ['X','Y','Z']
    fet_40 = dict()
    
    for i in col:
        fet_40['mean_' + i ] = mean(signal[i])
        fet_40['std_' + i ] = std_dev(signal[i])
        fet_40['mad_' + i ] = mad(signal[i])
        fet_40['max_' + i ] = max_value(signal[i])
        fet_40['min_' + i ] = min_value(signal[i])
        fet_40['energy_' + i ] = energy_measure(signal[i])
        fet_40['iqr_' + i ] = inter_quartile_range(signal[i])
        fet_40['entropy_' + i ] = entropy_axial(signal[i])
    
    
    
    counter = -1
    for count in range(1,13):
        index = 4 if count % 4  == 0 else count % 4 
        counter += 1 if index == 1 else 0
        i = col[counter]  
        
        arCoeff = t_arburg_axial(signal)
        fet_40[i + '_' + str(index)] = arCoeff[count-1]
          

      
        
    fet_40['sma'] = t_sma_axial(signal)
    fet_40['correlation_x_y'] = coor(signal)[0]
    fet_40['correlation_x_z'] = coor(signal)[1]
    fet_40['correlation_y_z'] = coor(signal)[2]
    
    res = {}
    for key in l:
        res[name+'_'+key] = fet_40[key]
    
    return res

def feature_13(signal, name):
    fet_13 = dict()
    
    fet_13[name + '_mean'] = mean(signal)
    fet_13[name + '_std'] = std_dev(signal)
    fet_13[name + '_mad'] = mad(signal)
    fet_13[name + '_max'] = max_value(signal)
    fet_13[name + '_min'] = min_value(signal)
    fet_13[name + '_sma'] = t_sma_mag(signal)
    fet_13[name + '_energy'] = energy_measure(signal)
    fet_13[name + '_iqr'] = inter_quartile_range(signal)
    fet_13[name + '_entropy'] = entropy_axial(signal)
    
    t_arburg= t_arburg_mag(signal)
    for count in range(1,5):
        fet_13[name + '_AR_' + str(count)] = t_arburg[count-1]
    
    return fet_13

def feature_80(signal, name):
    l = ['mean_X', 'mean_Y', 'mean_Z',
         'std_X','std_Y', 'std_Z',
         'mad_X','mad_Y', 'mad_Z',
         'max_X','max_Y', 'max_Z',
         'min_X','min_Y', 'min_Z',
         'sma',
         'energy_X', 'energy_Y','energy_Z',
         'iqr_X', 'iqr_Y', 'iqr_Z',
         'entropy_X', 'entropy_Y', 'entropy_Z',
         'maxInds_X', 'maxInds_Y', 'maxInds_Z',
         'meanFreq_X', 'meanFreq_Y', 'meanFreq_Z',
         'skewness_X', 'kurtosis_X', 'skewness_Y', 'kurtosis_Y', 'skewness_Z', 'kurtosis_Z',
         'bandpower_X-1-8', 'bandpower_X-9-16', 'bandpower_X-17-24', 'bandpower_X-25-32', 'bandpower_X-33-40', 
         'bandpower_X-41-48', 'bandpower_X-49-56', 'bandpower_X-57-64', 
         'bandpower_X-1-16', 'bandpower_X-17-32', 'bandpower_X-33-48', 'bandpower_X-49-64',
         'bandpower_X-1-24', 'bandpower_X-25-48',
         
         'bandpower_Y-1-8', 'bandpower_Y-9-16', 'bandpower_Y-17-24', 'bandpower_Y-25-32', 'bandpower_Y-33-40', 
         'bandpower_Y-41-48', 'bandpower_Y-49-56', 'bandpower_Y-57-64', 
         'bandpower_Y-1-16', 'bandpower_Y-17-32', 'bandpower_Y-33-48', 'bandpower_Y-49-64', 
         'bandpower_Y-1-24', 'bandpower_Y-25-48',
         
         'bandpower_Z-1-8', 'bandpower_Z-9-16', 'bandpower_Z-17-24', 'bandpower_Z-25-32', 'bandpower_Z-33-40', 
         'bandpower_Z-41-48', 'bandpower_Z-49-56', 'bandpower_Z-57-64', 
         'bandpower_Z-1-16', 'bandpower_Z-17-32', 'bandpower_Z-33-48', 'bandpower_Z-49-64', 
         'bandpower_Z-1-24', 'bandpower_Z-25-48' 
         ]

    col = ['X','Y','Z']
    fet_80 = dict()
    
    for i in col:
        fet_80['mean_' + i ] = mean(signal[i])
        fet_80['std_' + i ] = std_dev(signal[i])
        fet_80['mad_' + i ] = mad(signal[i])
        fet_80['max_' + i ] = max_value(signal[i])
        fet_80['min_' + i ] = min_value(signal[i])
        fet_80['energy_' + i ] = f_energy_axial(signal[i])
        fet_80['iqr_' + i ] = inter_quartile_range(signal[i])
        fet_80['entropy_' + i ] = entropy_axial(signal[i])
        
        fet_80['skewness_' + i] = skewness(signal[i])
        fet_80['kurtosis_' + i] = kurtosis(signal[i])

        for j in range(1,65,8):
            fet_80['bandpower_' + i + '-' + str(j) + '-' + str(j+7) ] = f_one_band_energy(signal.values, (j, j+7))
            
        for k in range(1,65,16):
            fet_80['bandpower_' + i + '-' + str(k) + '-' + str(k+15) ] = f_one_band_energy(signal.values, (k, k+15))

        for q in range(1,65,24):
            if q+23 > 65:break
            fet_80['bandpower_' + i + '-' + str(q) + '-' + str(q+23)] = f_one_band_energy(signal.values, (q, q+23))
    
    
    for i in range(3):
        fet_80['maxInds_' + col[i] ] = f_max_Inds_axial(signal)[i]
        fet_80['meanFreq_' + col[i] ] = f_mean_Freq_axial(signal)[i]
    
    fet_80['sma'] = f_sma_axial(signal)

    
    res = {}
    for key in l:
        res[name+'_'+key] = fet_80[key]
    
    return res


def feature_13_MAG(signal, name):
    fet_mag = dict()
    
    fet_mag[name + '_mean'] = mean(signal)
    fet_mag[name + '_std'] = std_dev(signal)
    fet_mag[name + '_mad'] = mad(signal)
    fet_mag[name + '_max'] = max_value(signal)
    fet_mag[name + '_min'] = min_value(signal)
    fet_mag[name + '_sma'] = f_sma_mag(signal)
    fet_mag[name + '_energy'] = f_energy_mag(signal)
    fet_mag[name + '_iqr'] = inter_quartile_range(signal)
    fet_mag[name + '_entropy'] = entropy(signal)
    
    fet_mag[name + '_maxInds' ] = f_max_Inds_mag(signal)
    fet_mag[name + '_meanFreq'] = f_mean_Freq_mag(signal)
    fet_mag[name + '_skewness'] = f_skewness_mag(signal)
    fet_mag[name + '_kurtosis'] = f_kurtosis_mag(signal)    
    
    return fet_mag


def last_features():
    feat = dict()
    
    V2_Vector=np.array(tGravityAcc.mean()) # mean values
    
    # angle 0: angle between (t_body_acc[X.mean,Y.mean,Z.mean], t_gravity[X.mean,Y.mean,Z.mean])
    V1_Vector=np.array(tBodyAcc[col].mean()) # mean values of t_body_acc[X,Y,Z]
    feat['tBodyAccMean_angle'] = angle(V1_Vector, V2_Vector) # angle between the vectors added to the global list
    
    # angle 1: (t_body_acc_jerk[X.mean,Y.mean,Z.mean],t_gravity[X.mean,Y.mean,Z.mean]
    V1_Vector=np.array(tBodyAccJerk[col].mean())
    feat['tBodyAccJerkMean_angle'] = angle(V1_Vector, V2_Vector)
    
    # angle 2: (t_body_gyro[X.mean,Y.mean,Z.mean],t_gravity[X.mean,Y.mean,Z.mean]
    V1_Vector=np.array(tBodyGyro[col].mean())
    feat['tBodyGyroMean_angle'] =  angle(V1_Vector, V2_Vector)
    
    # angle 3: (t_body_gyro_jerk[X.mean,Y.mean,Z.mean],t_gravity[X.mean,Y.mean,Z.mean]
    V1_Vector=np.array(tBodyGyroJerk[col].mean())
    feat['tBodyGyroJerkMean_angle'] = angle(V1_Vector, V2_Vector)
    
    
    # V1 vector in this case is the X axis itself [1,0,0]
    # angle 4: ([X_axis],t_gravity[X.mean,Y.mean,Z.mean])   
    V1_Vector=np.array([1,0,0])
    feat['tGravityAcc_X_angle'] = angle(V1_Vector, V2_Vector)
    
    # V1 vector in this case is the Y axis itself [0,1,0]
    # angle 5: ([Y_acc_axis],t_gravity[X.mean,Y.mean,Z.mean]) 
    V1_Vector=np.array([0,1,0])
    feat['tGravityAcc_Y_angle'] = angle(V1_Vector, V2_Vector)
    
    # V1 vector in this case is the Z axis itself [0,0,1]
    # angle 6: ([Z_acc_axis],t_gravity[X.mean,Y.mean,Z.mean])
    V1_Vector=np.array([0,0,1])
    feat['tGravityAcc_Z_angle'] = angle(V1_Vector, V2_Vector)
    
 
    return feat  

In [42]:
def collectFeatures():
    res = dict()
    
    feat40 = {'tBodyAcc':tBodyAcc,'tGravityAcc': tGravityAcc,'tBodyAccJerk':tBodyAccJerk,
              'tBodyGyro':tBodyGyro,'tBodyGyroJerk': tBodyGyroJerk} 
    for k,v in feat40.items():
        res.update(feature_40(v,k ))
    
    
    
    feat13 = {'tBodyAccMag': tBodyAccMag,'tGravityAccMag': tGravityAccMag,
              'tBodyAccJerkMag': tBodyAccJerkMag,'tBodyGyroMag': tBodyGyroMag,
              'tBodyGyroJerkMag':tBodyGyroJerkMag}
    for k,v in feat13.items():
        res.update(feature_13(v[k],k ))
    
    
    
    feat_80 = {'fBodyAcc': fBodyAcc, 'fBodyAccJerk': fBodyAcc, 'fBodyGyro': fBodyGyro}
    for k,v in feat_80.items():
        res.update(feature_80(v,k))
    
    
    
    feat13_mag = {'fBodyAccMag':fBodyAccMag,'fBodyAccJerkMag': fBodyAccJerkMag,
                  'fBodyGyroMag': fBodyGyroMag, 'fBodyGyroJerkMag': fBodyGyroJerkMag}
    for k,v in feat13_mag.items():
        res.update(feature_13_MAG(v[k],k ))
    
    res.update(last_features())
  
    return res

In [43]:
col_DF = pd.read_csv('Test_DATASET/test_data.csv').columns
path = 'Real-Time'

## Walking Up

In [ ]:
base_acc = pd.read_csv('{0}/walking_up_acc.csv'.format(path))
base_gryo = pd.read_csv('{0}/walking_up_gryo.csv'.format(path))

print(base_acc.shape, base_gryo.shape)


last = pd.DataFrame(columns=col_DF)
print(last.shape)

n = base_acc.shape[0]//129

for i in range(n):
    gryo = base_gryo.iloc[i*129:(i+1) * 129,:]
    acc = base_acc.iloc[i*129:(i+1) * 129,:]
    


    result = {}

    tBodyAcc, tGravityAcc, tBodyAccJerk, tBodyGyro, tBodyGyroJerk, \
    tBodyAccMag, tGravityAccMag,tBodyAccJerkMag, tBodyGyroMag, tBodyGyroJerkMag, \
    fBodyAcc, fBodyAccJerk, fBodyGyro, \
    fBodyAccMag, fBodyAccJerkMag, fBodyGyroMag,fBodyGyroJerkMag = main(gryo,acc)


    result = collectFeatures()
    result = pd.DataFrame(result, index=[0])

    last = pd.concat([last, result], axis = 0)


walking_up = last
walking_up.describe()

## Walking Down

In [ ]:
base_acc = pd.read_csv('{0}/walking_down_acc.csv'.format(path))
base_gryo = pd.read_csv('{0}/walking_down_gryo.csv'.format(path))

print(base_acc.shape, base_gryo.shape)

last = pd.DataFrame(columns=col_DF)
print(last.shape)

n = base_acc.shape[0]//129

for i in range(n):
    gryo = base_gryo.iloc[i*129:(i+1) * 129,:]
    acc = base_acc.iloc[i*129:(i+1) * 129,:]

    result = {}

    tBodyAcc, tGravityAcc, tBodyAccJerk, tBodyGyro, tBodyGyroJerk, \
    tBodyAccMag, tGravityAccMag,tBodyAccJerkMag, tBodyGyroMag, tBodyGyroJerkMag, \
    fBodyAcc, fBodyAccJerk, fBodyGyro, \
    fBodyAccMag, fBodyAccJerkMag, fBodyGyroMag,fBodyGyroJerkMag = main(gryo,acc)


    result = collectFeatures()
    result = pd.DataFrame(result, index=[0])

    last = pd.concat([last, result], axis = 0)




walking_down = last
print(walking_down.shape)
walking_down.describe()

## Walking

In [ ]:
base_acc = pd.read_csv('{0}/walking_acc.csv'.format(path))
base_gryo = pd.read_csv('{0}/walking_gryo.csv'.format(path))

print(base_acc.shape, base_gryo.shape)


last = pd.DataFrame(columns=col_DF)
print(last.shape)

n = base_acc.shape[0]//129

for i in range(n):
    gryo = base_gryo.iloc[i*129:(i+1) * 129,:]
    acc = base_acc.iloc[i*129:(i+1) * 129,:]

    result = {}

    tBodyAcc, tGravityAcc, tBodyAccJerk, tBodyGyro, tBodyGyroJerk, \
    tBodyAccMag, tGravityAccMag,tBodyAccJerkMag, tBodyGyroMag, tBodyGyroJerkMag, \
    fBodyAcc, fBodyAccJerk, fBodyGyro, \
    fBodyAccMag, fBodyAccJerkMag, fBodyGyroMag,fBodyGyroJerkMag = main(gryo,acc)


    result = collectFeatures()
    result = pd.DataFrame(result, index=[0])

    last = pd.concat([last, result], axis = 0)


walking = last
print(walking.shape)
walking.describe()

In [ ]:
def mag_3_signals(x,y,z): # Euclidian magnitude
    return [math.sqrt((x[i]**2+y[i]**2+z[i]**2)) for i in range(len(x))]
grav_acc_mag = mag_3_signals(tGravityAcc["X"], tGravityAcc['Y'], tGravityAcc['Z'])

print('mean value = '+str(np.array(grav_acc_mag).mean())[0:5]+ ' g')

## Sitting

In [ ]:
base_acc = pd.read_csv('{0}/sitting_acc.csv'.format(path))
base_gryo = pd.read_csv('{0}/sitting_gryo.csv'.format(path))

print(base_acc.shape, base_gryo.shape)



last = pd.DataFrame(columns=col_DF)
print(last.shape)

n = base_acc.shape[0]//129

for i in range(n):
    gryo = base_gryo.iloc[i*129:(i+1) * 129,:]
    acc = base_acc.iloc[i*129:(i+1) * 129,:]

    result = {}

    tBodyAcc, tGravityAcc, tBodyAccJerk, tBodyGyro, tBodyGyroJerk, \
    tBodyAccMag, tGravityAccMag,tBodyAccJerkMag, tBodyGyroMag, tBodyGyroJerkMag, \
    fBodyAcc, fBodyAccJerk, fBodyGyro, \
    fBodyAccMag, fBodyAccJerkMag, fBodyGyroMag,fBodyGyroJerkMag = main(gryo,acc)


    result = collectFeatures()
    result = pd.DataFrame(result, index=[0])

    last = pd.concat([last, result], axis = 0)


sitting = last
print(sitting.shape)
sitting.describe()

In [ ]:
base_acc = pd.read_csv('Real-Time/sitting_acc.csv')
base_gryo = pd.read_csv('Real-Time/sitting_gryo.csv')


base_acc.plot()

In [ ]:
base_acc = pd.read_csv('Real-Time_new/sitting_acc.csv')
base_gryo = pd.read_csv('Real-Time/sitting_gryo.csv')


base_acc.plot()

## Staying

In [ ]:
base_acc = pd.read_csv('{0}/staying_acc.csv'.format(path))
base_gryo = pd.read_csv('{0}/staying_gryo.csv'.format(path))

print(base_acc.shape, base_gryo.shape)


last = pd.DataFrame(columns=col_DF)
print(last.shape)

n = base_acc.shape[0]//129

for i in range(n):
    gryo = base_gryo.iloc[i*129:(i+1) * 129,:]
    acc = base_acc.iloc[i*129:(i+1) * 129,:]

    result = {}

    tBodyAcc, tGravityAcc, tBodyAccJerk, tBodyGyro, tBodyGyroJerk, \
    tBodyAccMag, tGravityAccMag,tBodyAccJerkMag, tBodyGyroMag, tBodyGyroJerkMag, \
    fBodyAcc, fBodyAccJerk, fBodyGyro, \
    fBodyAccMag, fBodyAccJerkMag, fBodyGyroMag,fBodyGyroJerkMag = main(gryo,acc)


    result = collectFeatures()
    result = pd.DataFrame(result, index=[0])

    last = pd.concat([last, result], axis = 0)


staying = last
print(staying.shape)
staying.describe()

## Laying

In [ ]:
base_acc = pd.read_csv('{0}/lying_acc.csv'.format(path))
base_gryo = pd.read_csv('{0}/lying_gryo.csv'.format(path))

print(base_acc.shape, base_gryo.shape)


last = pd.DataFrame(columns=col_DF)
print(last.shape)

n = base_acc.shape[0]//129

for i in range(n):
    gryo = base_gryo.iloc[i*129:(i+1) * 129,:]
    acc = base_acc.iloc[i*129:(i+1) * 129,:]

    result = {}

    tBodyAcc, tGravityAcc, tBodyAccJerk, tBodyGyro, tBodyGyroJerk, \
    tBodyAccMag, tGravityAccMag,tBodyAccJerkMag, tBodyGyroMag, tBodyGyroJerkMag, \
    fBodyAcc, fBodyAccJerk, fBodyGyro, \
    fBodyAccMag, fBodyAccJerkMag, fBodyGyroMag,fBodyGyroJerkMag = main(gryo,acc)


    result = collectFeatures()
    result = pd.DataFrame(result, index=[0])

    last = pd.concat([last, result], axis = 0)



lying = last
print(lying.shape)
lying.describe()

In [ ]:
LABELS = [
    "WALKING",
    "WALKING_UPSTAIRS",
    "WALKING_DOWNSTAIRS",
    "SITTING",
    "STANDING",
    "LAYING"
]

In [ ]:
X = pd.concat([walking, walking_up, walking_down, sitting, staying, lying],axis = 0)

In [ ]:
w = ['WALKING' for _ in range(walking.shape[0]) ]
up = ['WALKING_UPSTAIRS' for _ in range(walking_up.shape[0]) ]
down = ['WALKING_DOWNSTAIRS' for _ in range(walking_down.shape[0]) ]
sit = ['SITTING' for _ in range(sitting.shape[0]) ]
s = ['STANDING' for _ in range(staying.shape[0]) ]
lay = ['LAYING' for _ in range(lying.shape[0]) ]

In [ ]:
w = [1 for _ in range(walking.shape[0]) ]
up = [2 for _ in range(walking_up.shape[0]) ]
down = [3 for _ in range(walking_down.shape[0]) ]
sit = [4 for _ in range(sitting.shape[0]) ]
s = [5 for _ in range(staying.shape[0]) ]
lay = [6 for _ in range(lying.shape[0]) ]


Y =  w + up + down + sit + s + lay 
Y = pd.DataFrame(data = Y,columns=['Label'])

In [ ]:
X.to_csv('Real-Time/X.csv', index = None)
Y.to_csv('Real-Time/y.csv', index = None)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

X_test = X.copy()

scaler = MinMaxScaler(feature_range = [-1,1])
X_test  = scaler.fit_transform(X_test)

result = pd.DataFrame(data = X_test, columns= walking_up.columns)
result.to_csv('Real-Time/X_test.csv', index = None)

In [46]:
pd.options.display.max_columns = 9999
pd.options.display.float_format = '{:,.5f}'.format


In [ ]:
result

In [ ]:
result[['tGravityAcc_Z_angle']].plot()

In [ ]:
X.reset_index(inplace = True)

In [ ]:
X[['tGravityAcc_Z_angle']].plot()

In [57]:
base_acc = pd.read_csv('test_DATASET/ACC_TEST.csv',  delimiter = ',')
base_gryo = pd.read_csv('test_DATASET/GYRO_TEST.csv',  delimiter = ',')

print(base_acc.shape, base_gryo.shape)

(128, 3) (128, 3)


In [58]:
base_acc

,X,Y,Z
0,1.01282,-0.12322,0.10293
1,1.02283,-0.12688,0.10569
2,1.02203,-0.12400,0.10210
3,1.01788,-0.12493,0.10655
4,1.02368,-0.12577,0.10281
...,...,...,...
123,1.01981,-0.12701,0.09484
124,1.01929,-0.12618,0.09835
125,1.01845,-0.12407,0.10039
126,1.01937,-0.12275,0.09987


In [56]:
last = pd.DataFrame(columns=col_DF)
print(last.shape)


result = {}

tBodyAcc, tGravityAcc, tBodyAccJerk, tBodyGyro, tBodyGyroJerk, \
tBodyAccMag, tGravityAccMag,tBodyAccJerkMag, tBodyGyroMag, tBodyGyroJerkMag, \
fBodyAcc, fBodyAccJerk, fBodyGyro, \
fBodyAccMag, fBodyAccJerkMag, fBodyGyroMag,fBodyGyroJerkMag = main(base_gryo,base_acc)


result = collectFeatures()
result = pd.DataFrame(result, index=[0])

last = pd.concat([last, result], axis = 0)


test = last
print(test.shape)
test.describe()

(0, 561)
(1, 561)


,tBodyAcc_mean_X,tBodyAcc_mean_Y,tBodyAcc_mean_Z,tBodyAcc_std_X,tBodyAcc_std_Y,tBodyAcc_std_Z,tBodyAcc_mad_X,tBodyAcc_mad_Y,tBodyAcc_mad_Z,tBodyAcc_max_X,tBodyAcc_max_Y,tBodyAcc_max_Z,tBodyAcc_min_X,tBodyAcc_min_Y,tBodyAcc_min_Z,tBodyAcc_sma,tBodyAcc_energy_X,tBodyAcc_energy_Y,tBodyAcc_energy_Z,tBodyAcc_iqr_X,tBodyAcc_iqr_Y,tBodyAcc_iqr_Z,tBodyAcc_entropy_X,tBodyAcc_entropy_Y,tBodyAcc_entropy_Z,tBodyAcc_X_1,tBodyAcc_X_2,tBodyAcc_X_3,tBodyAcc_X_4,tBodyAcc_Y_1,tBodyAcc_Y_2,tBodyAcc_Y_3,tBodyAcc_Y_4,tBodyAcc_Z_1,tBodyAcc_Z_2,tBodyAcc_Z_3,tBodyAcc_Z_4,tBodyAcc_correlation_x_y,tBodyAcc_correlation_x_z,tBodyAcc_correlation_y_z,tGravityAcc_mean_X,tGravityAcc_mean_Y,tGravityAcc_mean_Z,tGravityAcc_std_X,tGravityAcc_std_Y,tGravityAcc_std_Z,tGravityAcc_mad_X,tGravityAcc_mad_Y,tGravityAcc_mad_Z,tGravityAcc_max_X,tGravityAcc_max_Y,tGravityAcc_max_Z,tGravityAcc_min_X,tGravityAcc_min_Y,tGravityAcc_min_Z,tGravityAcc_sma,tGravityAcc_energy_X,tGravityAcc_energy_Y,tGravityAcc_energy_Z,tGravityAcc_iqr_X,tGravityAcc_iqr_Y,tGravityAcc_iqr_Z,tGravityAcc_entropy_X,tGravityAcc_entropy_Y,tGravityAcc_entropy_Z,tGravityAcc_X_1,tGravityAcc_X_2,tGravityAcc_X_3,tGravityAcc_X_4,tGravityAcc_Y_1,tGravityAcc_Y_2,tGravityAcc_Y_3,tGravityAcc_Y_4,tGravityAcc_Z_1,tGravityAcc_Z_2,tGravityAcc_Z_3,tGravityAcc_Z_4,tGravityAcc_correlation_x_y,tGravityAcc_correlation_x_z,tGravityAcc_correlation_y_z,tBodyAccJerk_mean_X,tBodyAccJerk_mean_Y,tBodyAccJerk_mean_Z,tBodyAccJerk_std_X,tBodyAccJerk_std_Y,tBodyAccJerk_std_Z,tBodyAccJerk_mad_X,tBodyAccJerk_mad_Y,tBodyAccJerk_mad_Z,tBodyAccJerk_max_X,tBodyAccJerk_max_Y,tBodyAccJerk_max_Z,tBodyAccJerk_min_X,tBodyAccJerk_min_Y,tBodyAccJerk_min_Z,tBodyAccJerk_sma,tBodyAccJerk_energy_X,tBodyAccJerk_energy_Y,tBodyAccJerk_energy_Z,tBodyAccJerk_iqr_X,tBodyAccJerk_iqr_Y,tBodyAccJerk_iqr_Z,tBodyAccJerk_entropy_X,tBodyAccJerk_entropy_Y,tBodyAccJerk_entropy_Z,tBodyAccJerk_X_1,tBodyAccJerk_X_2,tBodyAccJerk_X_3,tBodyAccJerk_X_4,tBodyAccJerk_Y_1,tBodyAccJerk_Y_2,tBodyAccJerk_Y_3,tBodyAccJerk_Y_4,tBodyAccJerk_Z_1,tBodyAccJerk_Z_2,tBodyAccJerk_Z_3,tBodyAccJerk_Z_4,tBodyAccJerk_correlation_x_y,tBodyAccJerk_correlation_x_z,tBodyAccJerk_correlation_y_z,tBodyGyro_mean_X,tBodyGyro_mean_Y,tBodyGyro_mean_Z,tBodyGyro_std_X,tBodyGyro_std_Y,tBodyGyro_std_Z,tBodyGyro_mad_X,tBodyGyro_mad_Y,tBodyGyro_mad_Z,tBodyGyro_max_X,tBodyGyro_max_Y,tBodyGyro_max_Z,tBodyGyro_min_X,tBodyGyro_min_Y,tBodyGyro_min_Z,tBodyGyro_sma,tBodyGyro_energy_X,tBodyGyro_energy_Y,tBodyGyro_energy_Z,tBodyGyro_iqr_X,tBodyGyro_iqr_Y,tBodyGyro_iqr_Z,tBodyGyro_entropy_X,tBodyGyro_entropy_Y,tBodyGyro_entropy_Z,tBodyGyro_X_1,tBodyGyro_X_2,tBodyGyro_X_3,tBodyGyro_X_4,tBodyGyro_Y_1,tBodyGyro_Y_2,tBodyGyro_Y_3,tBodyGyro_Y_4,tBodyGyro_Z_1,tBodyGyro_Z_2,tBodyGyro_Z_3,tBodyGyro_Z_4,tBodyGyro_correlation_x_y,tBodyGyro_correlation_x_z,tBodyGyro_correlation_y_z,tBodyGyroJerk_mean_X,tBodyGyroJerk_mean_Y,tBodyGyroJerk_mean_Z,tBodyGyroJerk_std_X,tBodyGyroJerk_std_Y,tBodyGyroJerk_std_Z,tBodyGyroJerk_mad_X,tBodyGyroJerk_mad_Y,tBodyGyroJerk_mad_Z,tBodyGyroJerk_max_X,tBodyGyroJerk_max_Y,tBodyGyroJerk_max_Z,tBodyGyroJerk_min_X,tBodyGyroJerk_min_Y,tBodyGyroJerk_min_Z,tBodyGyroJerk_sma,tBodyGyroJerk_energy_X,tBodyGyroJerk_energy_Y,tBodyGyroJerk_energy_Z,tBodyGyroJerk_iqr_X,tBodyGyroJerk_iqr_Y,tBodyGyroJerk_iqr_Z,tBodyGyroJerk_entropy_X,tBodyGyroJerk_entropy_Y,tBodyGyroJerk_entropy_Z,tBodyGyroJerk_X_1,tBodyGyroJerk_X_2,tBodyGyroJerk_X_3,tBodyGyroJerk_X_4,tBodyGyroJerk_Y_1,tBodyGyroJerk_Y_2,tBodyGyroJerk_Y_3,tBodyGyroJerk_Y_4,tBodyGyroJerk_Z_1,tBodyGyroJerk_Z_2,tBodyGyroJerk_Z_3,tBodyGyroJerk_Z_4,tBodyGyroJerk_correlation_x_y,tBodyGyroJerk_correlation_x_z,tBodyGyroJerk_correlation_y_z,tBodyAccMag_mean,tBodyAccMag_std,tBodyAccMag_mad,tBodyAccMag_max,tBodyAccMag_min,tBodyAccMag_sma,tBodyAccMag_energy,tBodyAccMag_iqr,tBodyAccMag_entropy,tBodyAccMag_AR_1,tBodyAccMag_AR_2,tBodyAccMag_AR_3,tBodyAccMag_AR_4,tGravityAccMag_mean,tGravityAccMag_std,tGravityAccMag_mad,tGravityAccMag_max,tGravityAccMag_min,tGravityAccMag_sma,tGra

In [54]:
test

,tBodyAcc_mean_X,tBodyAcc_mean_Y,tBodyAcc_mean_Z,tBodyAcc_std_X,tBodyAcc_std_Y,tBodyAcc_std_Z,tBodyAcc_mad_X,tBodyAcc_mad_Y,tBodyAcc_mad_Z,tBodyAcc_max_X,tBodyAcc_max_Y,tBodyAcc_max_Z,tBodyAcc_min_X,tBodyAcc_min_Y,tBodyAcc_min_Z,tBodyAcc_sma,tBodyAcc_energy_X,tBodyAcc_energy_Y,tBodyAcc_energy_Z,tBodyAcc_iqr_X,tBodyAcc_iqr_Y,tBodyAcc_iqr_Z,tBodyAcc_entropy_X,tBodyAcc_entropy_Y,tBodyAcc_entropy_Z,tBodyAcc_X_1,tBodyAcc_X_2,tBodyAcc_X_3,tBodyAcc_X_4,tBodyAcc_Y_1,tBodyAcc_Y_2,tBodyAcc_Y_3,tBodyAcc_Y_4,tBodyAcc_Z_1,tBodyAcc_Z_2,tBodyAcc_Z_3,tBodyAcc_Z_4,tBodyAcc_correlation_x_y,tBodyAcc_correlation_x_z,tBodyAcc_correlation_y_z,tGravityAcc_mean_X,tGravityAcc_mean_Y,tGravityAcc_mean_Z,tGravityAcc_std_X,tGravityAcc_std_Y,tGravityAcc_std_Z,tGravityAcc_mad_X,tGravityAcc_mad_Y,tGravityAcc_mad_Z,tGravityAcc_max_X,tGravityAcc_max_Y,tGravityAcc_max_Z,tGravityAcc_min_X,tGravityAcc_min_Y,tGravityAcc_min_Z,tGravityAcc_sma,tGravityAcc_energy_X,tGravityAcc_energy_Y,tGravityAcc_energy_Z,tGravityAcc_iqr_X,tGravityAcc_iqr_Y,tGravityAcc_iqr_Z,tGravityAcc_entropy_X,tGravityAcc_entropy_Y,tGravityAcc_entropy_Z,tGravityAcc_X_1,tGravityAcc_X_2,tGravityAcc_X_3,tGravityAcc_X_4,tGravityAcc_Y_1,tGravityAcc_Y_2,tGravityAcc_Y_3,tGravityAcc_Y_4,tGravityAcc_Z_1,tGravityAcc_Z_2,tGravityAcc_Z_3,tGravityAcc_Z_4,tGravityAcc_correlation_x_y,tGravityAcc_correlation_x_z,tGravityAcc_correlation_y_z,tBodyAccJerk_mean_X,tBodyAccJerk_mean_Y,tBodyAccJerk_mean_Z,tBodyAccJerk_std_X,tBodyAccJerk_std_Y,tBodyAccJerk_std_Z,tBodyAccJerk_mad_X,tBodyAccJerk_mad_Y,tBodyAccJerk_mad_Z,tBodyAccJerk_max_X,tBodyAccJerk_max_Y,tBodyAccJerk_max_Z,tBodyAccJerk_min_X,tBodyAccJerk_min_Y,tBodyAccJerk_min_Z,tBodyAccJerk_sma,tBodyAccJerk_energy_X,tBodyAccJerk_energy_Y,tBodyAccJerk_energy_Z,tBodyAccJerk_iqr_X,tBodyAccJerk_iqr_Y,tBodyAccJerk_iqr_Z,tBodyAccJerk_entropy_X,tBodyAccJerk_entropy_Y,tBodyAccJerk_entropy_Z,tBodyAccJerk_X_1,tBodyAccJerk_X_2,tBodyAccJerk_X_3,tBodyAccJerk_X_4,tBodyAccJerk_Y_1,tBodyAccJerk_Y_2,tBodyAccJerk_Y_3,tBodyAccJerk_Y_4,tBodyAccJerk_Z_1,tBodyAccJerk_Z_2,tBodyAccJerk_Z_3,tBodyAccJerk_Z_4,tBodyAccJerk_correlation_x_y,tBodyAccJerk_correlation_x_z,tBodyAccJerk_correlation_y_z,tBodyGyro_mean_X,tBodyGyro_mean_Y,tBodyGyro_mean_Z,tBodyGyro_std_X,tBodyGyro_std_Y,tBodyGyro_std_Z,tBodyGyro_mad_X,tBodyGyro_mad_Y,tBodyGyro_mad_Z,tBodyGyro_max_X,tBodyGyro_max_Y,tBodyGyro_max_Z,tBodyGyro_min_X,tBodyGyro_min_Y,tBodyGyro_min_Z,tBodyGyro_sma,tBodyGyro_energy_X,tBodyGyro_energy_Y,tBodyGyro_energy_Z,tBodyGyro_iqr_X,tBodyGyro_iqr_Y,tBodyGyro_iqr_Z,tBodyGyro_entropy_X,tBodyGyro_entropy_Y,tBodyGyro_entropy_Z,tBodyGyro_X_1,tBodyGyro_X_2,tBodyGyro_X_3,tBodyGyro_X_4,tBodyGyro_Y_1,tBodyGyro_Y_2,tBodyGyro_Y_3,tBodyGyro_Y_4,tBodyGyro_Z_1,tBodyGyro_Z_2,tBodyGyro_Z_3,tBodyGyro_Z_4,tBodyGyro_correlation_x_y,tBodyGyro_correlation_x_z,tBodyGyro_correlation_y_z,tBodyGyroJerk_mean_X,tBodyGyroJerk_mean_Y,tBodyGyroJerk_mean_Z,tBodyGyroJerk_std_X,tBodyGyroJerk_std_Y,tBodyGyroJerk_std_Z,tBodyGyroJerk_mad_X,tBodyGyroJerk_mad_Y,tBodyGyroJerk_mad_Z,tBodyGyroJerk_max_X,tBodyGyroJerk_max_Y,tBodyGyroJerk_max_Z,tBodyGyroJerk_min_X,tBodyGyroJerk_min_Y,tBodyGyroJerk_min_Z,tBodyGyroJerk_sma,tBodyGyroJerk_energy_X,tBodyGyroJerk_energy_Y,tBodyGyroJerk_energy_Z,tBodyGyroJerk_iqr_X,tBodyGyroJerk_iqr_Y,tBodyGyroJerk_iqr_Z,tBodyGyroJerk_entropy_X,tBodyGyroJerk_entropy_Y,tBodyGyroJerk_entropy_Z,tBodyGyroJerk_X_1,tBodyGyroJerk_X_2,tBodyGyroJerk_X_3,tBodyGyroJerk_X_4,tBodyGyroJerk_Y_1,tBodyGyroJerk_Y_2,tBodyGyroJerk_Y_3,tBodyGyroJerk_Y_4,tBodyGyroJerk_Z_1,tBodyGyroJerk_Z_2,tBodyGyroJerk_Z_3,tBodyGyroJerk_Z_4,tBodyGyroJerk_correlation_x_y,tBodyGyroJerk_correlation_x_z,tBodyGyroJerk_correlation_y_z,tBodyAccMag_mean,tBodyAccMag_std,tBodyAccMag_mad,tBodyAccMag_max,tBodyAccMag_min,tBodyAccMag_sma,tBodyAccMag_energy,tBodyAccMag_iqr,tBodyAccMag_entropy,tBodyAccMag_AR_1,tBodyAccMag_AR_2,tBodyAccMag_AR_3,tBodyAccMag_AR_4,tGravityAccMag_mean,tGravityAccMag_std,tGravityAccMag_mad,tGravityAccMag_max,tGravityAccMag_min,tGravityAccMag_sma,tGra

In [64]:
X_train_fex = pd.read_csv("UCI HAR Dataset/train/X_train.txt", delim_whitespace=True, header=None, names = col_DF)

In [65]:
pd.concat([X_train_fex[:1]])

,tBodyAcc_mean_X,tBodyAcc_mean_Y,tBodyAcc_mean_Z,tBodyAcc_std_X,tBodyAcc_std_Y,tBodyAcc_std_Z,tBodyAcc_mad_X,tBodyAcc_mad_Y,tBodyAcc_mad_Z,tBodyAcc_max_X,tBodyAcc_max_Y,tBodyAcc_max_Z,tBodyAcc_min_X,tBodyAcc_min_Y,tBodyAcc_min_Z,tBodyAcc_sma,tBodyAcc_energy_X,tBodyAcc_energy_Y,tBodyAcc_energy_Z,tBodyAcc_iqr_X,tBodyAcc_iqr_Y,tBodyAcc_iqr_Z,tBodyAcc_entropy_X,tBodyAcc_entropy_Y,tBodyAcc_entropy_Z,tBodyAcc_X_1,tBodyAcc_X_2,tBodyAcc_X_3,tBodyAcc_X_4,tBodyAcc_Y_1,tBodyAcc_Y_2,tBodyAcc_Y_3,tBodyAcc_Y_4,tBodyAcc_Z_1,tBodyAcc_Z_2,tBodyAcc_Z_3,tBodyAcc_Z_4,tBodyAcc_correlation_x_y,tBodyAcc_correlation_x_z,tBodyAcc_correlation_y_z,tGravityAcc_mean_X,tGravityAcc_mean_Y,tGravityAcc_mean_Z,tGravityAcc_std_X,tGravityAcc_std_Y,tGravityAcc_std_Z,tGravityAcc_mad_X,tGravityAcc_mad_Y,tGravityAcc_mad_Z,tGravityAcc_max_X,tGravityAcc_max_Y,tGravityAcc_max_Z,tGravityAcc_min_X,tGravityAcc_min_Y,tGravityAcc_min_Z,tGravityAcc_sma,tGravityAcc_energy_X,tGravityAcc_energy_Y,tGravityAcc_energy_Z,tGravityAcc_iqr_X,tGravityAcc_iqr_Y,tGravityAcc_iqr_Z,tGravityAcc_entropy_X,tGravityAcc_entropy_Y,tGravityAcc_entropy_Z,tGravityAcc_X_1,tGravityAcc_X_2,tGravityAcc_X_3,tGravityAcc_X_4,tGravityAcc_Y_1,tGravityAcc_Y_2,tGravityAcc_Y_3,tGravityAcc_Y_4,tGravityAcc_Z_1,tGravityAcc_Z_2,tGravityAcc_Z_3,tGravityAcc_Z_4,tGravityAcc_correlation_x_y,tGravityAcc_correlation_x_z,tGravityAcc_correlation_y_z,tBodyAccJerk_mean_X,tBodyAccJerk_mean_Y,tBodyAccJerk_mean_Z,tBodyAccJerk_std_X,tBodyAccJerk_std_Y,tBodyAccJerk_std_Z,tBodyAccJerk_mad_X,tBodyAccJerk_mad_Y,tBodyAccJerk_mad_Z,tBodyAccJerk_max_X,tBodyAccJerk_max_Y,tBodyAccJerk_max_Z,tBodyAccJerk_min_X,tBodyAccJerk_min_Y,tBodyAccJerk_min_Z,tBodyAccJerk_sma,tBodyAccJerk_energy_X,tBodyAccJerk_energy_Y,tBodyAccJerk_energy_Z,tBodyAccJerk_iqr_X,tBodyAccJerk_iqr_Y,tBodyAccJerk_iqr_Z,tBodyAccJerk_entropy_X,tBodyAccJerk_entropy_Y,tBodyAccJerk_entropy_Z,tBodyAccJerk_X_1,tBodyAccJerk_X_2,tBodyAccJerk_X_3,tBodyAccJerk_X_4,tBodyAccJerk_Y_1,tBodyAccJerk_Y_2,tBodyAccJerk_Y_3,tBodyAccJerk_Y_4,tBodyAccJerk_Z_1,tBodyAccJerk_Z_2,tBodyAccJerk_Z_3,tBodyAccJerk_Z_4,tBodyAccJerk_correlation_x_y,tBodyAccJerk_correlation_x_z,tBodyAccJerk_correlation_y_z,tBodyGyro_mean_X,tBodyGyro_mean_Y,tBodyGyro_mean_Z,tBodyGyro_std_X,tBodyGyro_std_Y,tBodyGyro_std_Z,tBodyGyro_mad_X,tBodyGyro_mad_Y,tBodyGyro_mad_Z,tBodyGyro_max_X,tBodyGyro_max_Y,tBodyGyro_max_Z,tBodyGyro_min_X,tBodyGyro_min_Y,tBodyGyro_min_Z,tBodyGyro_sma,tBodyGyro_energy_X,tBodyGyro_energy_Y,tBodyGyro_energy_Z,tBodyGyro_iqr_X,tBodyGyro_iqr_Y,tBodyGyro_iqr_Z,tBodyGyro_entropy_X,tBodyGyro_entropy_Y,tBodyGyro_entropy_Z,tBodyGyro_X_1,tBodyGyro_X_2,tBodyGyro_X_3,tBodyGyro_X_4,tBodyGyro_Y_1,tBodyGyro_Y_2,tBodyGyro_Y_3,tBodyGyro_Y_4,tBodyGyro_Z_1,tBodyGyro_Z_2,tBodyGyro_Z_3,tBodyGyro_Z_4,tBodyGyro_correlation_x_y,tBodyGyro_correlation_x_z,tBodyGyro_correlation_y_z,tBodyGyroJerk_mean_X,tBodyGyroJerk_mean_Y,tBodyGyroJerk_mean_Z,tBodyGyroJerk_std_X,tBodyGyroJerk_std_Y,tBodyGyroJerk_std_Z,tBodyGyroJerk_mad_X,tBodyGyroJerk_mad_Y,tBodyGyroJerk_mad_Z,tBodyGyroJerk_max_X,tBodyGyroJerk_max_Y,tBodyGyroJerk_max_Z,tBodyGyroJerk_min_X,tBodyGyroJerk_min_Y,tBodyGyroJerk_min_Z,tBodyGyroJerk_sma,tBodyGyroJerk_energy_X,tBodyGyroJerk_energy_Y,tBodyGyroJerk_energy_Z,tBodyGyroJerk_iqr_X,tBodyGyroJerk_iqr_Y,tBodyGyroJerk_iqr_Z,tBodyGyroJerk_entropy_X,tBodyGyroJerk_entropy_Y,tBodyGyroJerk_entropy_Z,tBodyGyroJerk_X_1,tBodyGyroJerk_X_2,tBodyGyroJerk_X_3,tBodyGyroJerk_X_4,tBodyGyroJerk_Y_1,tBodyGyroJerk_Y_2,tBodyGyroJerk_Y_3,tBodyGyroJerk_Y_4,tBodyGyroJerk_Z_1,tBodyGyroJerk_Z_2,tBodyGyroJerk_Z_3,tBodyGyroJerk_Z_4,tBodyGyroJerk_correlation_x_y,tBodyGyroJerk_correlation_x_z,tBodyGyroJerk_correlation_y_z,tBodyAccMag_mean,tBodyAccMag_std,tBodyAccMag_mad,tBodyAccMag_max,tBodyAccMag_min,tBodyAccMag_sma,tBodyAccMag_energy,tBodyAccMag_iqr,tBodyAccMag_entropy,tBodyAccMag_AR_1,tBodyAccMag_AR_2,tBodyAccMag_AR_3,tBodyAccMag_AR_4,tGravityAccMag_mean,tGravityAccMag_std,tGravityAccMag_mad,tGravityAccMag_max,tGravityAccMag_min,tGravityAccMag_sma,tGra

In [68]:
base_gryo.describe()

,X,Y,Z
count,128.00000,128.00000,128.00000
mean,0.02225,0.01940,0.00947
std,0.01449,0.01991,0.00597
min,-0.00386,-0.00902,-0.00689
25%,0.01118,0.00379,0.00567
50%,0.01806,0.01419,0.00892
75%,0.03317,0.03051,0.01397
max,0.05054,0.07485,0.02286


In [67]:
pd.concat([test,X_train_fex[:1]])

,tBodyAcc_mean_X,tBodyAcc_mean_Y,tBodyAcc_mean_Z,tBodyAcc_std_X,tBodyAcc_std_Y,tBodyAcc_std_Z,tBodyAcc_mad_X,tBodyAcc_mad_Y,tBodyAcc_mad_Z,tBodyAcc_max_X,tBodyAcc_max_Y,tBodyAcc_max_Z,tBodyAcc_min_X,tBodyAcc_min_Y,tBodyAcc_min_Z,tBodyAcc_sma,tBodyAcc_energy_X,tBodyAcc_energy_Y,tBodyAcc_energy_Z,tBodyAcc_iqr_X,tBodyAcc_iqr_Y,tBodyAcc_iqr_Z,tBodyAcc_entropy_X,tBodyAcc_entropy_Y,tBodyAcc_entropy_Z,tBodyAcc_X_1,tBodyAcc_X_2,tBodyAcc_X_3,tBodyAcc_X_4,tBodyAcc_Y_1,tBodyAcc_Y_2,tBodyAcc_Y_3,tBodyAcc_Y_4,tBodyAcc_Z_1,tBodyAcc_Z_2,tBodyAcc_Z_3,tBodyAcc_Z_4,tBodyAcc_correlation_x_y,tBodyAcc_correlation_x_z,tBodyAcc_correlation_y_z,tGravityAcc_mean_X,tGravityAcc_mean_Y,tGravityAcc_mean_Z,tGravityAcc_std_X,tGravityAcc_std_Y,tGravityAcc_std_Z,tGravityAcc_mad_X,tGravityAcc_mad_Y,tGravityAcc_mad_Z,tGravityAcc_max_X,tGravityAcc_max_Y,tGravityAcc_max_Z,tGravityAcc_min_X,tGravityAcc_min_Y,tGravityAcc_min_Z,tGravityAcc_sma,tGravityAcc_energy_X,tGravityAcc_energy_Y,tGravityAcc_energy_Z,tGravityAcc_iqr_X,tGravityAcc_iqr_Y,tGravityAcc_iqr_Z,tGravityAcc_entropy_X,tGravityAcc_entropy_Y,tGravityAcc_entropy_Z,tGravityAcc_X_1,tGravityAcc_X_2,tGravityAcc_X_3,tGravityAcc_X_4,tGravityAcc_Y_1,tGravityAcc_Y_2,tGravityAcc_Y_3,tGravityAcc_Y_4,tGravityAcc_Z_1,tGravityAcc_Z_2,tGravityAcc_Z_3,tGravityAcc_Z_4,tGravityAcc_correlation_x_y,tGravityAcc_correlation_x_z,tGravityAcc_correlation_y_z,tBodyAccJerk_mean_X,tBodyAccJerk_mean_Y,tBodyAccJerk_mean_Z,tBodyAccJerk_std_X,tBodyAccJerk_std_Y,tBodyAccJerk_std_Z,tBodyAccJerk_mad_X,tBodyAccJerk_mad_Y,tBodyAccJerk_mad_Z,tBodyAccJerk_max_X,tBodyAccJerk_max_Y,tBodyAccJerk_max_Z,tBodyAccJerk_min_X,tBodyAccJerk_min_Y,tBodyAccJerk_min_Z,tBodyAccJerk_sma,tBodyAccJerk_energy_X,tBodyAccJerk_energy_Y,tBodyAccJerk_energy_Z,tBodyAccJerk_iqr_X,tBodyAccJerk_iqr_Y,tBodyAccJerk_iqr_Z,tBodyAccJerk_entropy_X,tBodyAccJerk_entropy_Y,tBodyAccJerk_entropy_Z,tBodyAccJerk_X_1,tBodyAccJerk_X_2,tBodyAccJerk_X_3,tBodyAccJerk_X_4,tBodyAccJerk_Y_1,tBodyAccJerk_Y_2,tBodyAccJerk_Y_3,tBodyAccJerk_Y_4,tBodyAccJerk_Z_1,tBodyAccJerk_Z_2,tBodyAccJerk_Z_3,tBodyAccJerk_Z_4,tBodyAccJerk_correlation_x_y,tBodyAccJerk_correlation_x_z,tBodyAccJerk_correlation_y_z,tBodyGyro_mean_X,tBodyGyro_mean_Y,tBodyGyro_mean_Z,tBodyGyro_std_X,tBodyGyro_std_Y,tBodyGyro_std_Z,tBodyGyro_mad_X,tBodyGyro_mad_Y,tBodyGyro_mad_Z,tBodyGyro_max_X,tBodyGyro_max_Y,tBodyGyro_max_Z,tBodyGyro_min_X,tBodyGyro_min_Y,tBodyGyro_min_Z,tBodyGyro_sma,tBodyGyro_energy_X,tBodyGyro_energy_Y,tBodyGyro_energy_Z,tBodyGyro_iqr_X,tBodyGyro_iqr_Y,tBodyGyro_iqr_Z,tBodyGyro_entropy_X,tBodyGyro_entropy_Y,tBodyGyro_entropy_Z,tBodyGyro_X_1,tBodyGyro_X_2,tBodyGyro_X_3,tBodyGyro_X_4,tBodyGyro_Y_1,tBodyGyro_Y_2,tBodyGyro_Y_3,tBodyGyro_Y_4,tBodyGyro_Z_1,tBodyGyro_Z_2,tBodyGyro_Z_3,tBodyGyro_Z_4,tBodyGyro_correlation_x_y,tBodyGyro_correlation_x_z,tBodyGyro_correlation_y_z,tBodyGyroJerk_mean_X,tBodyGyroJerk_mean_Y,tBodyGyroJerk_mean_Z,tBodyGyroJerk_std_X,tBodyGyroJerk_std_Y,tBodyGyroJerk_std_Z,tBodyGyroJerk_mad_X,tBodyGyroJerk_mad_Y,tBodyGyroJerk_mad_Z,tBodyGyroJerk_max_X,tBodyGyroJerk_max_Y,tBodyGyroJerk_max_Z,tBodyGyroJerk_min_X,tBodyGyroJerk_min_Y,tBodyGyroJerk_min_Z,tBodyGyroJerk_sma,tBodyGyroJerk_energy_X,tBodyGyroJerk_energy_Y,tBodyGyroJerk_energy_Z,tBodyGyroJerk_iqr_X,tBodyGyroJerk_iqr_Y,tBodyGyroJerk_iqr_Z,tBodyGyroJerk_entropy_X,tBodyGyroJerk_entropy_Y,tBodyGyroJerk_entropy_Z,tBodyGyroJerk_X_1,tBodyGyroJerk_X_2,tBodyGyroJerk_X_3,tBodyGyroJerk_X_4,tBodyGyroJerk_Y_1,tBodyGyroJerk_Y_2,tBodyGyroJerk_Y_3,tBodyGyroJerk_Y_4,tBodyGyroJerk_Z_1,tBodyGyroJerk_Z_2,tBodyGyroJerk_Z_3,tBodyGyroJerk_Z_4,tBodyGyroJerk_correlation_x_y,tBodyGyroJerk_correlation_x_z,tBodyGyroJerk_correlation_y_z,tBodyAccMag_mean,tBodyAccMag_std,tBodyAccMag_mad,tBodyAccMag_max,tBodyAccMag_min,tBodyAccMag_sma,tBodyAccMag_energy,tBodyAccMag_iqr,tBodyAccMag_entropy,tBodyAccMag_AR_1,tBodyAccMag_AR_2,tBodyAccMag_AR_3,tBodyAccMag_AR_4,tGravityAccMag_mean,tGravityAccMag_std,tGravityAccMag_mad,tGravityAccMag_max,tGravityAccMag_min,tGravityAccMag_sma,tGra

In [ ]:
base_gr

In [ ]:
def mag_3_signals(x,y,z): # Euclidian magnitude
    return [math.sqrt((x[i]**2+y[i]**2+z[i]**2)) for i in range(len(x))]
grav_acc_mag = mag_3_signals(tGravityAcc["X"], tGravityAcc['Y'], tGravityAcc['Z'])

print('mean value = '+str(np.array(grav_acc_mag).mean())[0:5]+ ' g')

In [ ]:
from sklearn.preprocessing import MinMaxScaler

X_test = test.copy()

scaler = MinMaxScaler(feature_range = [-1,1])
X_test  = scaler.fit_transform(X_test)

result = pd.DataFrame(data = X_test, columns= walking_up.columns)
result.to_csv('Real-Time/X_test.csv', index = None)

In [ ]:
base_gryo.describe()

In [ ]:
base_acc.describe()